In [2]:
##這支程式在把Amazon購物網站的槌子商品資訊網頁的htmL從mongo撈出來然後用Beautifulsoup解析

from bs4 import BeautifulSoup
import requests
import re
import pymongo  
from pymongo import MongoClient
import pyodbc
import time
from datetime import datetime
import csv
import MongoConnect
headers = {
    'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"
}  

In [3]:
#從Mongo把html撈出來解析，用mongo_data變數存起來，會花一點時間
db=MongoConnect.mongoconnect()
one_document=db.Amazon_reviews_hammer_detail_html
list=one_document.find({ 'error':0 })
mongo_data=[]
print('Ready to Insert')
loop=1
for i in list:
    if (loop%1000)==0:
        print(loop)
    mongo_data.append(i)
    loop=loop+1

Ready to Insert
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [144]:
num=1
error=0

#對每個HTML去解析商品的資訊
for row in mongo_data:
    try:
        url=row['url']
        web_html=row['web_html']
        tool_name=None
        art_title=None
        tool_star=None
        tool_price=None
        
        soup1=BeautifulSoup(web_html,'lxml')
        #抓取槌子的資訊
        tool_name=soup1.find('a',id='bylineInfo').text
        art_title=soup1.find('span',id="productTitle").text.strip()
        tool_star=soup1.find('span',id="acrPopover").find('span',class_='a-icon-alt').text
        tool_price=soup1.find('span',id='priceblock_ourprice').text

        ####存入bag

        bag={
            'url':url,
            'tool_name':tool_name,
            'art_title':art_title,
            'tool_star':tool_star,
            'tool_price':tool_price,
            'error':0
        }
        #這個網頁有一個html表格存放槌子的規格資訊，解析表格然後給bag變數存起來，bag是一個dictionary的結構
        table=soup1.find('table',id='productDetails_techSpec_section_1')
        th=table.find_all('th')
        td=table.find_all('td')
        for i in range(0,len(th)):
            bag[th[i].text.strip().lower()]=td[i].text.strip()
        ####
    except Exception as e:
        print(e)
        if tool_name is None:tool_name=''
        if art_title is None:art_title=''
        if tool_star is None:tool_star=''
        if tool_price is None:tool_price=''
        
        bag={
            'url':url,
            'tool_name':tool_name,
            'art_title':art_title,
            'tool_star':tool_star,
            'tool_price':tool_price,
            'error':1
        }
    finally:
        
        ###連接MONGO



        ##檢查Mongo看有沒有抓過該hammer的產品資訊的html
        one_document =db.Amazon_reviews_hammer_detail
        current_insert=one_document.find({'url':url}).count()
        if(current_insert==0):
            one_document.insert_one(bag)
            print('num:'+str(num)+'\nhammer工具:'+url)
            if(bag['error']):print('\n爬取此頁面發生問題')
            num=num+1
        if(current_insert>0):
            print('num:'+str(num)+'\nhammer工具:'+url)
            print("此工具已抓過了")

        



'NoneType' object has no attribute 'text'
num:1
hammer工具:https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_tools_sr_pg1_1?ie=UTF8&adId=A0739756334V95BUR4XL&url=https%3A%2F%2Fwww.amazon.com%2FConstruction-Rubberized-Contractors-Automotive-Maintenance%2Fdp%2FB01MS1KOEA%2Fref%3Dsr_1_1_sspa%2F147-6603142-7052802%3Fs%3Dpower-hand-tools%26ie%3DUTF8%26qid%3D1529051023%26sr%3D1-1-spons%26keywords%3Dhammer%26psc%3D1&qualifier=1529051023&id=2140523781816837&widgetName=sp_atf

爬取此頁面發生問題
num:2
hammer工具:https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_tools_sr_pg1_2?ie=UTF8&adId=A03638461856QVEFO57PW&url=https%3A%2F%2Fwww.amazon.com%2FOverpeak-8-Ounce-Christmas-Birthday-Fathers%2Fdp%2FB01MTYQHTH%2Fref%3Dsr_1_2_sspa%2F147-6603142-7052802%3Fs%3Dpower-hand-tools%26ie%3DUTF8%26qid%3D1529051023%26sr%3D1-2-spons%26keywords%3Dhammer%26psc%3D1&qualifier=1529051023&id=2140523781816837&widgetName=sp_atf
'NoneType' object has no attribute 'text'
num:3
hammer工具:http

num:34
hammer工具:https://www.amazon.com/11436-Ounce-Premium-Hickory-Autograf/dp/B00002N5KB/ref=sr_1_53/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051815&sr=1-53&keywords=hammer
num:35
hammer工具:https://www.amazon.com/Original-Pink-Box-PB12HM-12-Ounce/dp/B002DMLJMQ/ref=sr_1_54/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051815&sr=1-54&keywords=hammer
num:36
hammer工具:https://www.amazon.com/Naturehike-Camping-Outdoor-Aluminum-Multi-function/dp/B00N4RLG54/ref=sr_1_55/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051815&sr=1-55&keywords=hammer
num:37
hammer工具:https://www.amazon.com/Powerbuilt-648332-2-5-Pound-Hammer-Fiberglass/dp/B000FPAMFS/ref=sr_1_56/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051815&sr=1-56&keywords=hammer
num:38
hammer工具:https://www.amazon.com/Fiskars-IsoCore-Pound-Club-Hammer/dp/B01CDAC2K0/ref=sr_1_57/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051815&sr=1-57&keywords=hammer
num:39
hammer工具:https://www.amazon.com/Estw

'NoneType' object has no attribute 'text'
num:71
hammer工具:https://www.amazon.com/Estwing-E3-16C-Hammer-Metal-Handle/dp/B00002N5NA/ref=sr_1_85/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051822&sr=1-85&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:72
hammer工具:https://www.amazon.com/Fiskars-IsoCore-Milled-face-Framing-Hammer/dp/B01CDDLYYW/ref=sr_1_86/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051822&sr=1-86&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'find_all'
num:73
hammer工具:https://www.amazon.com/Seymour-407-19-16-Inch-Hammer-Handle/dp/B000H1B7PE/ref=sr_1_87/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051822&sr=1-87&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:74
hammer工具:https://www.amazon.com/Great-Neck-HG16S-Ripping-Fiberglass/dp/B000FP8E2G/ref=sr_1_88/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051822&sr=1-88&keywords=hammer

爬取此頁面發生問題
num:75
hammer工具:https://www.a

num:106
hammer工具:https://www.amazon.com/TEKTON-31103-Jacketed-Fiberglass-3-Pound/dp/B00IMMI6RW/ref=sr_1_115/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051828&sr=1-115&keywords=hammer
num:107
hammer工具:https://www.amazon.com/Apollo-Tools-DT0240-Screwdrivers-Adjustable/dp/B001CEEL3K/ref=sr_1_116/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051828&sr=1-116&keywords=hammer
'NoneType' object has no attribute 'text'
num:108
hammer工具:https://www.amazon.com/Stanley-51-713-Hickory-Handle-Nailing/dp/B00002X1XA/ref=sr_1_117/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051828&sr=1-117&keywords=hammer

爬取此頁面發生問題
num:109
hammer工具:https://www.amazon.com/Vaughan-Bushnell-1650-Dalluge-Hickory/dp/B00100DY84/ref=sr_1_118/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051828&sr=1-118&keywords=hammer
num:110
hammer工具:https://www.amazon.com/Vaughan-CF2HC-19-Ounce-California-Hickory/dp/B00004Z2X7/ref=sr_1_119/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051828

'NoneType' object has no attribute 'find'
num:142
hammer工具:https://www.amazon.com/Mini-Hammer-Screwdriver-Steel-Wood-Tools/dp/B07BR316ZZ/ref=sr_1_146/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529051835&sr=1-146&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:143
hammer工具:https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_tools_sr_pg6_2?ie=UTF8&adId=A02297921J4N4F0P28SF8&url=https%3A%2F%2Fwww.amazon.com%2FPersonalized-Engraved-Wood-Fathers-Anniversary%2Fdp%2FB01H4838AY%2Fref%3Dsr_1_147_sspa%2F147-6603142-7052802%3Fs%3Dpower-hand-tools%26ie%3DUTF8%26qid%3D1529051835%26sr%3D1-147-spons%26keywords%3Dhammer%26psc%3D1&qualifier=1529051835&id=1505003777867051&widgetName=sp_btf

爬取此頁面發生問題
'NoneType' object has no attribute 'find'
num:144
hammer工具:https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_tools_sr_pg6_1?ie=UTF8&adId=A091546412MC3HU4ZXA1P&url=https%3A%2F%2Fwww.amazon.com%2FTopBuilt-Ounce-Hammer-Tubular-Handle%2

...

'NoneType' object has no attribute 'find'
num:10238
hammer工具:https://www.amazon.com/Jackhammer-Hose-Assemblies-0-38-I-D/dp/B07BK7HBPW/ref=sr_1_9413/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059144&sr=1-9413&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'find'
num:10239
hammer工具:https://www.amazon.com/Jackhammer-Hose-Assemblies-0-38-I-D/dp/B07BK7BFDT/ref=sr_1_9414/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059144&sr=1-9414&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'find'
num:10240
hammer工具:https://www.amazon.com/Jackhammer-Hose-Assemblies-0-38-I-D/dp/B07BK756MF/ref=sr_1_9416/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059144&sr=1-9416&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:10241
hammer工具:https://www.amazon.com/Emergency-niceeshop-Multi-function-Camping-Screwdriver/dp/B07BK41W92/ref=sr_1_9417/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059144&sr=1-9417&keywords=hammer

爬取此

num:10275
hammer工具:https://www.amazon.com/10-Ounce-Birthday-Daughter-Personalized/dp/B07B7DWZXQ/ref=sr_1_9451/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059151&sr=1-9451&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:10276
hammer工具:https://www.amazon.com/Atlas-Welding-Accessories-Inc-Hammer/dp/B0774HT4XS/ref=sr_1_9452/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059151&sr=1-9452&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'find'
num:10277
hammer工具:https://www.amazon.com/Bearing-Extractor-Hammer-Internal-Remover/dp/B07B7C26HR/ref=sr_1_9453/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059151&sr=1-9453&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:10278
hammer工具:https://www.amazon.com/Heyco-2165003500-Plastic-hammer-2165/dp/B077W5DK3C/ref=sr_1_9454/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059151&sr=1-9454&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'find'


'NoneType' object has no attribute 'find'
num:10310
hammer工具:https://www.amazon.com/Chipping-Hammers-Chisel-Steel-Handle/dp/B07B6FLN79/ref=sr_1_9488/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059164&sr=1-9488&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:10311
hammer工具:https://www.amazon.com/Chipping-Hammers-Chisel-Steel-Handle/dp/B07B6FL2CM/ref=sr_1_9489/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059164&sr=1-9489&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:10312
hammer工具:https://www.amazon.com/Chipping-Hammers-Chisel-Steel-Handle/dp/B07B6FGJYC/ref=sr_1_9490/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059164&sr=1-9490&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:10313
hammer工具:https://www.amazon.com/Chipping-Hammers-Chisel-Black-Steel/dp/B07B6FD9RP/ref=sr_1_9491/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059164&sr=1-9491&keywords=hammer

爬取此頁面發生問題
'NoneType

'NoneType' object has no attribute 'find'
num:10345
hammer工具:https://www.amazon.com/Estwing-Framing-Hammer-Straight-Handle/dp/B076VSY7F6/ref=sr_1_9527/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059171&sr=1-9527&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:10346
hammer工具:https://www.amazon.com/Estwing-Ripping-Hammer-Straight-Handle/dp/B012U7FEQK/ref=sr_1_9528/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059171&sr=1-9528&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:10347
hammer工具:https://www.amazon.com/gp/help/customer/display.html/ref=sr_hms_help/147-6603142-7052802?nodeId=468556&ie=UTF8&qid=1529059171

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:10348
hammer工具:https://www.amazon.com/Proferred-12oz-Hickory-Ball-Pein-Hammer/dp/B07B1T5PN2/ref=sr_1_9529/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059178&sr=1-9529&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:10349

'NoneType' object has no attribute 'find'
num:10379
hammer工具:https://www.amazon.com/Jackson-Double-Sledge-Hammers-Hickory/dp/B079TCRW8H/ref=sr_1_9561/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059185&sr=1-9561&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'find'
num:10380
hammer工具:https://www.amazon.com/Jackson-Double-Sledge-Hammers-Hickory/dp/B079TCQN1P/ref=sr_1_9562/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059185&sr=1-9562&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:10381
hammer工具:https://www.amazon.com/Jackson-Double-Sledge-Hammers-Fiberglass/dp/B079TCQFJQ/ref=sr_1_9563/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059185&sr=1-9563&keywords=hammer

爬取此頁面發生問題
'NoneType' object has no attribute 'text'
num:10382
hammer工具:https://www.amazon.com/Jackson-Double-Sledge-Hammers-Hickory/dp/B079TCQFGY/ref=sr_1_9564/147-6603142-7052802?s=power-hand-tools&ie=UTF8&qid=1529059185&sr=1-9564&keywords=hammer

爬取此頁面發生問題
'

In [7]:
##把一些沒爬成功的欄位再爬一次
num=1

for row in mongo_data:
    try:
        bag={}
        _id=row['_id']
        url=row['url']
        web_html=row['web_html']
        soup1=BeautifulSoup(web_html,'lxml')
        
        #這個網頁有一個html表格存放槌子的規格資訊，解析表格然後給bag變數存起來，bag是一個dictionary的結構
        table=soup1.find('table',id='productDetails_techSpec_section_1')
        th=table.find_all('th')
        td=table.find_all('td')
        
        
        for i in range(0,len(th)):
            bag[th[i].text.strip().lower()]=td[i].text.strip()
        
        
        ###連接MONGO


        ##Update Mongo
        
        one_document =db.Amazon_reviews_hammer_detail
        
        one_document.update_one({"_id":_id},{"$set":bag}) 
        num=num+1
        print(str(num))
    except Exception as e:
        print(e)
        
  
        
      



2
3
4
5
'NoneType' object has no attribute 'find_all'
6
7
8
9
10
11
'NoneType' object has no attribute 'find_all'
12
13
14
15
16
17
18
19
'NoneType' object has no attribute 'find_all'
20
21
22
23
'NoneType' object has no attribute 'find_all'
24
25
'NoneType' object has no attribute 'find_all'
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
'NoneType' object has no attribute 'find_all'
54
55
56
57
58
59
60
61
62
63
64
65
66
67
'NoneType' object has no attribute 'find_all'
68
69
70
71
72
73
74
75
76
77
78
79
80
'NoneType' object has no attribute 'find_all'
81
82
83
84
85
86
87
'NoneType' object has no attribute 'find_all'
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
'NoneType' object has no attribute 'find_all'
106
'NoneType' object has no attribute 'find_all'
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
'NoneType' object has no attribute 'find_all'
135
136
137
'NoneType'

732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
'NoneType' object has no attribute 'find_all'
750
751
752
753
'NoneType' object has no attribute 'find_all'
754
755
756
757
758
759
760
761
'NoneType' object has no attribute 'find_all'
762
'NoneType' object has no attribute 'find_all'
763
764
765
'NoneType' object has no attribute 'find_all'
766
767
768
769
770
'NoneType' object has no attribute 'find_all'
771
772
773
774
'NoneType' object has no attribute 'find_all'
775
776
'NoneType' object has no attribute 'find_all'
777
778
'NoneType' object has no attribute 'find_all'
779
780
781
782
783
784
785
'NoneType' object has no attribute 'find_all'
786
787
788
789
790
791
792
'NoneType' object has no attribute 'find_all'
793
794
795
796
'NoneType' object has no attribute 'find_all'
797
798
799
800
'NoneType' object has no attribute 'find_all'
801
802
803
804
805
806
807
808
809
'NoneType' object has no attribute 'find_all'
810
'NoneType' object has no attribute 'find

917
'NoneType' object has no attribute 'find_all'
918
919
'NoneType' object has no attribute 'find_all'
920
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
921
922
'NoneType' object has no attribute 'find_all'
923
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
924
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
925
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all

999
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
1000
1001
1002
'NoneType' object has no attribute 'find_all'
1003
1004
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
1005
1006
'NoneType' object has no attribute 'find_all'
1007
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
1008
1009
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
1010
'NoneType' object has no attribute 'find_all'
1011
'NoneType' objec

1071
1072
'NoneType' object has no attribute 'find_all'
1073
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
1074
1075
'NoneType' object has no attribute 'find_all'
1076
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
1077
1078
'NoneType' object has no attribute 'find_all'
1079
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
1080
'NoneType' object has no attribute 'find_all'
1081
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
1082
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
1083
'NoneType' object has no attribute 'find_all'
1084
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType'

...

'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute

'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
2787
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attr

2823
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
2824
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
2825
2826
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' obj

'NoneType' object has no attribute 'find_all'
2860
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
2861
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
2862
2863
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
2864
'NoneType' object has no attribute 'find_all'
2865
2866
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'

2924
'NoneType' object has no attribute 'find_all'
2925
2926
'NoneType' object has no attribute 'find_all'
2927
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' obj